In [1]:
from tqdm import tqdm
import os
import json

In [2]:
from glob import glob

files = glob('mixtral-rag-question-factually-wrong/*.json')

In [3]:
questions = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    output = data[-1]
    splitted = output.strip().split('\n')
    splitted = [s for s in splitted if '.' if s and '?' in s]
    splitted = ['.'.join(s.split('.')[1:]).strip() for s in splitted]
    splitted = [s for s in splitted if len(s) > 3]
    splitted = [s[1:] if s[0] == '"' else s for s in splitted]
    q = [s[:-1] if s[-1] == '"' else s for s in splitted]
    q = [(data[0], s) for s in q if len(s) > 10]
    questions.extend(q)
    
# questions = list(set(questions))
len(questions)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 91268/91268 [00:03<00:00, 26118.18it/s]


813203

In [4]:
exists = set()
actual_questions = []
for q in tqdm(questions):
    if q[1] not in exists:
        exists.add(q[1])
        actual_questions.append(q)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 813203/813203 [00:00<00:00, 3056633.22it/s]


In [5]:
len(actual_questions)

801581

In [6]:
actual_questions[0]

('Last weekend,\xa0Realme just had their first ever Realme 3 sale roadshow. It showcases all of the Realme products that are available in the market – like the Realme 3 which we love and also merchandises like the\xa0earbuds, tumbler, bagpacks and many more.\nSince the sale roadshow is held in a mall,\xa0visitors also won cash prizes up to RM1,000 by competing in a mini mobile game tournament at the roadshow.\n\n\n\nIt happened at two different places –\xa0Sunway Pyramid Shopping Mall and Sunway Velocity Mall. At these two venues, there were fans from all ages and from multiple different knowledge of Realme itself. Here are a few highlights from the Realme fans.\nDharani a/p Kaliappan\nDharani a/p Kaliappan, a 23-year-old Realme fan and now an owner of the Realme 3 smartphone has said that she liked the specs and design of the phone, and can’t believe that she’s able to own one for herself. The price is one of the main highlights on why she chooses this phone.\nLim Tiak Cheong\nNext we

In [7]:
files = [
    'maktabahalbakri.com.jsonl',
    'muftiwp.gov.my.jsonl',
    'asklegal.jsonl',
    'dewanbahasa-jdbp.jsonl',
    'gov.my.jsonl',
]

In [8]:
def partition(text, size = 2000):
    splitted = text.split()
    return [' '.join(splitted[i: i + size]) for i in range(0, len(splitted), size) if len(splitted[i: i + size]) > 500]

In [9]:
import math

all_texts = {}
for f in files:
    texts = []
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            if isinstance(l, dict):
                l = l['body']
            if len(l.split()) < 150:
                continue
            all_texts[l] = f
            
len(all_texts)

33102

In [10]:
not_count = {'maktabahalbakri.com.jsonl', 'muftiwp.gov.my.jsonl'}

In [11]:
from collections import defaultdict

questions = {}
all_texts_count = defaultdict(int)
f_count = defaultdict(int)
for q in actual_questions:
    if q[0] not in all_texts:
        continue
    if all_texts[q[0]] in not_count and all_texts_count[q[0]] > 7:
        continue
    if all_texts[q[0]] not in not_count and all_texts_count[q[0]] > 2:
        continue
    questions[q[1]] = (all_texts[q[0]], q[0])
    all_texts_count[q[0]] += 1
    f_count[all_texts[q[0]]] += 1
    
f_count

defaultdict(int,
            {'dewanbahasa-jdbp.jsonl': 12368,
             'gov.my.jsonl': 25268,
             'muftiwp.gov.my.jsonl': 26251,
             'maktabahalbakri.com.jsonl': 26243,
             'asklegal.jsonl': 3690})

In [12]:
with open('filtered-factually-wrong-qa.jsonl', 'w') as fopen_l:
    with open('factually-wrong-qa.jsonl') as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            if l['question'] not in questions:
                continue
            
            d = {
                'question': l['question'],
                'answer': l['answer'],
                'context': questions[l['question']][1],
                'file': questions[l['question']][0],
            }
            fopen_l.write(f'{json.dumps(d)}\n')
            fopen_l.flush()

1108037it [00:29, 37396.73it/s]


In [13]:
!wc -l filtered-factually-wrong-qa.jsonl

82671 filtered-factually-wrong-qa.jsonl


In [16]:
!ls -lh filtered-factually-wrong-qa.jsonl

-rw-rw-r-- 1 husein husein 738M Jun  22 18:26 filtered-factually-wrong-qa.jsonl
